In [ ]:
# Header Files
import PyPDF2
import spacy
import csv
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import tabula
import numpy as np
from collections import Counter
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from wordcloud import WordCloud

## Budget Speech Files

In [ ]:
import requests

temp_url='https://www.indiabudget.gov.in/doc/bspeech/bs'

for i in range(1947,2019):
    add_text=str(i) + str(i+1)[2:] +".pdf"
    url= temp_url + add_text
    print(url)

    page = requests.get(url) # get url
    
    name = url.split('/')[-1] # to get filename
    f = open("budget_speeches/" + name,'wb')  # make a file object
    
    f.write(page.content) # write data
    f.close() 

### Ministry Budget Provision Files

In [ ]:

temp_url="https://www.indiabudget.gov.in/budget_archive/ub"    
    

for i in range(2000,2020):
#    if i<2017:
    add_text=str(i) + '-' + str(i+1)[2:] +"/eb/sumsbe.pdf"
#    else:
#        add_text=str(i) + '-' + str(i+1)[2:] +"/eb/stat1.pdf"
    url= temp_url + add_text
    print(url)

    page = requests.get(url) # get url
    
    f = open("ministry_budget_provision/es" + str(i) + '-' + str(i+1)[2:] + ".pdf",'wb')  # make a file object
    
    f.write(page.content) # write data
    f.close() 

In [ ]:
def table_to_df_1(year):
    print(year)

    file = "ministry_budget_provision/es" + str(year) + '-' + str(year+1)[2:] + ".pdf"
    tables = tabula.read_pdf(file, pages = "all")
    tabula.convert_into(file, "expenditure_summary/dummy_table.csv",pages="all")
    df=pd.read_csv("expenditure_summary/dummy_table.csv",names=["Ministry", "Plan Revenue", "Plan Capital", "Plan Total","Non Plan Revenue", "Non Plan Capital", "Non Plan Total", "Total","Page No."])
    df.dropna(subset=['Ministry'],inplace=True)
    df=df.loc[df['Ministry'].str.isupper()]

    cols=list(df.columns.drop("Ministry"))

    NAN_list=list(df.loc[df[cols].isnull().all(axis=1)].index)

    for n_index in NAN_list:

        try:
            df.loc[n_index+1]['Ministry']=df.loc[n_index]['Ministry'] + " " + df.loc[n_index+1]['Ministry']
        except:
            df.loc[n_index-1]['Ministry']=df.loc[n_index-1]['Ministry'] + " " + df.loc[n_index]['Ministry']

    df.drop(NAN_list,inplace=True)
    df.reset_index(inplace=True)

 
    df['Total Temp']=np.where(df['Total'].isnull() ,df['Non Plan Total'],df['Total'])
    df['Non Plan Total']=np.where(df['Total'].isnull() ,df['Non Plan Capital'].str.split().str[1],df['Non Plan Total'])
    df['Non Plan Capital']=np.where(df['Total'].isnull() ,df['Non Plan Capital'].str.split().str[1],df['Non Plan Capital'])
    df['Total']=df['Total Temp']

    df.loc[45:47]=df.loc[45:47].shift(periods=-1, axis='columns')
    df.loc[45:47]['Ministry']=['UNION TERRITORIES WITHOUT LEGISLATURE', 'RAILWAYS**','GRAND TOTAL']
    df.drop(['index','Page No.','Total Temp'],1,inplace=True)

    
    df['Year']=str(year) + '-' + str(year+1)[2:]
    return df


# table_to_df_1 for 2001


In [ ]:
def table_to_df_2(year):

    print(year)
    file = "ministry_budget_provision/es" + str(year) + '-' + str(year+1)[2:] + ".pdf"
    tables = tabula.read_pdf(file, pages = "all")
    tabula.convert_into(file, "expenditure_summary/dummy_table.csv",pages="all")
    df=pd.read_csv("expenditure_summary/dummy_table.csv",encoding= 'ISO-8859-1',names=["Ministry","Revenue", "Capital", "Total","Page No."],skiprows=6)
    df.dropna(subset=['Ministry'],inplace=True)
    df.drop(df.loc[~df['Ministry'].str.isupper()].index, inplace=True)
    df.reset_index(inplace=True)
    
    cols=list(df.columns.drop("Ministry"))
    
    NAN_list=list(df.loc[df[cols].isnull().all(axis=1)].index)
    
    for n_index in NAN_list:
        print(n_index)
        try:
            df.loc[n_index+1]['Ministry']=df.loc[n_index]['Ministry'] + " " + df.loc[n_index+1]['Ministry']
        except:
            df.loc[n_index-1]['Ministry']=df.loc[n_index-1]['Ministry'] + " " + df.loc[n_index]['Ministry']
            
    df.drop(NAN_list,inplace=True)

    # Cleaning 1

    df['Revenue'] = np.where(~df['Page No.'].isnull(),  df['Total'].str.split().str[0], df['Revenue'])
    df['Capital'] = np.where(~df['Page No.'].isnull(),  df['Total'].str.split().str[1], df['Capital'])
    df['Total'] = np.where(~df['Page No.'].isnull(),  df['Page No.'], df['Total'])


    # Cleaning 2
    df['Total_Temp'] = np.where(df['Total'].isnull(),  df['Capital'], '0')
    df['Capital'] = np.where(df['Total'].isnull(),  df['Revenue'].str.split().str[1], df['Capital'])
    df['Revenue'] = np.where(df['Total'].isnull(),  df['Revenue'].str.split().str[0], df['Revenue'])
    df['Total'] = np.where(df['Total'].isnull(),  df['Total_Temp'], df['Total'])
    df.drop(['Total_Temp'],1,inplace=True)

    # Cleaning 3
    df['Revenue_Temp'] = np.where(df['Revenue'].isnull(), df['Capital'].str.split().str[0], '0')
    df['Capital'] = np.where(df['Revenue'].isnull(),  df['Capital'].str.split().str[1], df['Capital'])
    df['Revenue'] = np.where(df['Revenue'].isnull(),  df['Revenue_Temp'], df['Revenue'])
    df.drop(['Revenue_Temp'],1,inplace=True)
    
    df.drop(['Page No.','index'],1,inplace=True)

    df['Year']=str(year) + '-' + str(year+1)[2:]
    
    
#     For Vice President
    df.loc[38]['Ministry']=df.loc[38]['Ministry'] + " " + df.loc[39]['Ministry']
    df.drop(39, inplace=True)
    df.reset_index(inplace=True)

    
    return df


# table_to_df_2 for 2017,2018,2019


In [ ]:
def table_to_df_3(year):
    print(year)
    file = "ministry_budget_provision/es" + str(year) + '-' + str(year+1)[2:] + ".pdf"
    tables = tabula.read_pdf(file, pages = "all")
    tabula.convert_into(file, "expenditure_summary/dummy_table.csv",pages="all")
    df=pd.read_csv("expenditure_summary/dummy_table.csv",encoding= 'ISO-8859-1',names=["Ministry", "Plan Revenue", "Plan Capital", "Plan Total","Non Plan Revenue", "Non Plan Capital", "Non Plan Total", "Total","Page No."])

    df.dropna(subset=['Ministry'],inplace=True)
    df.drop(df.loc[~df['Ministry'].str.isupper()].index, inplace=True)

    df.reset_index(inplace=True)
    
    cols=list(df.columns.drop("Ministry"))
    df[cols]=df[cols].shift(periods=-1, axis='columns')

    NAN_list=list(df.loc[df[cols].isnull().all(axis=1)].index)

    for n_index in NAN_list:
            try:
                df.loc[n_index+1]['Ministry']=df.loc[n_index]['Ministry'] + " " + df.loc[n_index+1]['Ministry']
            except:
                df.loc[n_index-1]['Ministry']=df.loc[n_index-1]['Ministry'] + " " + df.loc[n_index]['Ministry']

    df.drop(NAN_list,inplace=True)
    df.drop(['Page No.','index'],1,inplace=True)
    
    df['Year']=str(year) + '-' + str(year+1)[2:]
    
    return df 


# table_to_df_3 for 2016,2014,2012,2011

In [ ]:
def table_to_df_4(year):
    print(year)
    
    file = "ministry_budget_provision/es" + str(year) + '-' + str(year+1)[2:] + ".pdf"
    tables = tabula.read_pdf(file, pages = "all")
    tabula.convert_into(file, "expenditure_summary/dummy_table.csv",pages="all")
    df=pd.read_csv("expenditure_summary/dummy_table.csv",encoding= 'ISO-8859-1',names=["Ministry", "Plan Revenue", "Plan Capital", "Plan Total","Non Plan Revenue", "Non Plan Capital", "Non Plan Total", "Total","Page No."],skiprows=3)


    req_index=list(df[df['Plan Revenue'].str.isupper() & ~df['Plan Revenue'].isnull()].index)

    for index in req_index:
        df.loc[index] =df.loc[index].shift(periods=-1)


    df.dropna(subset=['Ministry'],inplace=True)
    df.drop(df.loc[~df['Ministry'].str.isupper()].index, inplace=True)


    cols=list(df.columns.drop("Ministry"))
    NAN_list=list(df.loc[df[cols].isnull().all(axis=1)].index)

    for n_index in NAN_list:
                try:
                    df.loc[n_index+1]['Ministry']=df.loc[n_index]['Ministry'] + " " + df.loc[n_index+1]['Ministry']
                except:
                    df.loc[n_index-1]['Ministry']=df.loc[n_index-1]['Ministry'] + " " + df.loc[n_index]['Ministry']


    df.drop(NAN_list,inplace=True)
    df.reset_index(inplace=True)
    df.drop(['index'],1,inplace=True)

    df['PC Len']=0
    for element in df['Plan Capital']:
        df['PC Len'][df['Plan Capital']==element]=len(element.split())

    # Cleaning 1

    df['Total']=np.where(df['PC Len']==3,df['Non Plan Total'],df['Total'])
    df['Non Plan Total']=np.where(df['PC Len']==3,df['Non Plan Capital'],df['Non Plan Total'])
    df['Non Plan Capital']=np.where(df['PC Len']==3,df['Non Plan Revenue'],df['Non Plan Capital'])

    df['Plan Total']=np.where(df['PC Len']==3,df['Plan Capital'].str.split().str[1],df['Plan Total'])
    df['Non Plan Revenue']=np.where(df['PC Len']==3,df['Plan Capital'].str.split().str[2],df['Non Plan Revenue'])
    df['Plan Capital']=np.where(df['PC Len']==3,df['Plan Capital'].str.split().str[0],df['Plan Capital'])

    # Cleaning 2

    df['Total']=np.where((df['PC Len']==2) & (~df['Non Plan Revenue'].isnull()),df['Non Plan Total'],df['Total'])
    df['Non Plan Total']=np.where((df['PC Len']==2) & (~df['Non Plan Revenue'].isnull()),df['Non Plan Capital'],df['Non Plan Total'])
    df['Non Plan Capital']=np.where((df['PC Len']==2) & (~df['Non Plan Revenue'].isnull()),df['Non Plan Revenue'],df['Non Plan Capital'])
    df['Non Plan Revenue']=np.where((df['PC Len']==2) & (~df['Non Plan Revenue'].isnull()),df['Plan Total'],df['Non Plan Revenue'])


    df['Plan Total']=np.where((df['PC Len']==2) & (~df['Non Plan Revenue'].isnull()),df['Plan Capital'].str.split().str[1],df['Plan Total'])
    df['Plan Capital']=np.where((df['PC Len']==2) & (~df['Non Plan Revenue'].isnull()),df['Plan Capital'].str.split().str[0],df['Plan Capital'])


    df['Non Plan Revenue Temp']=np.where((df['PC Len']==2) & (df['Non Plan Revenue'].isnull()),df['Plan Total'],df['Non Plan Revenue'])
    df['Plan Total']=np.where((df['PC Len']==2) & (df['Non Plan Revenue'].isnull()),df['Plan Capital'].str.split().str[1],df['Plan Total'])
    df['Plan Capital']=np.where((df['PC Len']==2) & (df['Non Plan Revenue'].isnull()),df['Plan Capital'].str.split().str[0],df['Plan Capital'])
    df['Non Plan Revenue']=df['Non Plan Revenue Temp']
    df.drop('Non Plan Revenue Temp',1,inplace=True)

    # Cleaning 3

    df['Plan Revenue']=np.where(df['PC Len']==1,df['Plan Capital'],df['Plan Revenue'])
    df['Non Plan Revenue']=np.where((df['PC Len']==1) & (df['Non Plan Revenue'].isnull()),df['Plan Total'].str.split().str[2],df['Non Plan Revenue'])
    df['Plan Capital']=np.where(df['PC Len']==1,df['Plan Total'].str.split().str[0],df['Plan Capital'])
    df['Plan Total']=np.where(df['PC Len']==1,df['Plan Total'].str.split().str[1],df['Plan Total'])



    df['Non Plan Capital']=np.where(df['PC Len']==1,df['Non Plan Total'],df['Non Plan Capital'])
    df['Non Plan Total']=np.where(df['PC Len']==1,df['Total'],df['Non Plan Total'])
    df['Total']=np.where(df['PC Len']==1,df['Page No.'],df['Total'])


    df['Total'].loc[54]=df['Non Plan Total'].loc[54].split()[1]
    df['Non Plan Total'].loc[54]=df['Non Plan Total'].loc[54].split()[0]

    df.drop(['Page No.','PC Len'],1,inplace=True)
    
    df['Year']=str(year) + '-' + str(year+1)[2:]
    
    return df

# table_to_df_3 for 2009,2010

In [ ]:
def table_to_df_5(year):
    print(year)
    file = "ministry_budget_provision/es" + str(year) + '-' + str(year+1)[2:] + ".pdf"
    tables = tabula.read_pdf(file, pages = "all")
    tabula.convert_into(file, "expenditure_summary/dummy_table.csv",pages="all")
    df=pd.read_csv("expenditure_summary/dummy_table.csv",encoding= 'ISO-8859-1',names=["Ministry", "Plan Revenue", "Plan Capital", "Plan Total","Non Plan Revenue", "Non Plan Capital", "Non Plan Total", "Total","Page No."],skiprows=3)


    req_index=list(df[df['Plan Revenue'].str.isupper() & ~df['Plan Revenue'].isnull()].index)

    for index in req_index:
        df.loc[index] =df.loc[index].shift(periods=-1)


    df.dropna(subset=['Ministry'],inplace=True)
    df.drop(df.loc[~df['Ministry'].str.isupper()].index, inplace=True)


    cols=list(df.columns.drop("Ministry"))
    NAN_list=list(df.loc[df[cols].isnull().all(axis=1)].index)

    for n_index in NAN_list:
                try:
                    df.loc[n_index+1]['Ministry']=df.loc[n_index]['Ministry'] + " " + df.loc[n_index+1]['Ministry']
                except:
                    df.loc[n_index-1]['Ministry']=df.loc[n_index-1]['Ministry'] + " " + df.loc[n_index]['Ministry']


    df.drop(NAN_list,inplace=True)
    df.reset_index(inplace=True)
    df.drop(['index'],1,inplace=True)
    
    df['Total']=np.where( (~df['Non Plan Capital'].isnull()),df['Non Plan Total'],df['Total'])
    df['Non Plan Total']=np.where((~df['Non Plan Capital'].isnull()),df['Non Plan Capital'],df['Non Plan Total'])
    df['Non Plan Capital']=np.where((~df['Non Plan Capital'].isnull()),df['Non Plan Revenue'],df['Non Plan Capital'])
    df['Non Plan Revenue']=np.where((~df['Non Plan Capital'].isnull()),df['Plan Total'],df['Non Plan Revenue'])


    df['Plan Total']=np.where((~df['Non Plan Capital'].isnull()),df['Plan Capital'].str.split().str[1],df['Plan Total'])
    df['Plan Capital']=np.where((~df['Non Plan Capital'].isnull()),df['Plan Capital'].str.split().str[0],df['Plan Capital'])

    df['Total']=np.where((df['Non Plan Capital'].isnull()),df['Non Plan Total'],df['Total'])
    df['Non Plan Total']=np.where((df['Non Plan Capital'].isnull()),df['Non Plan Revenue'].str.split().str[1],df['Non Plan Total'])
    df['Non Plan Capital Temp']=np.where((df['Non Plan Capital'].isnull()),df['Non Plan Revenue'].str.split().str[0],df['Non Plan Capital'])
    df['Non Plan Revenue']=np.where((df['Non Plan Capital'].isnull()),df['Plan Total'],df['Non Plan Revenue'])
    df['Plan Total']=np.where((df['Non Plan Capital'].isnull()),df['Plan Capital'].str.split().str[1],df['Plan Total'])
    df['Plan Capital']=np.where((df['Non Plan Capital'].isnull()),df['Plan Capital'].str.split().str[0],df['Plan Capital'])


    df['Non Plan Capital']=df['Non Plan Capital Temp']
    df.drop(['Non Plan Capital Temp','Page No.'],1,inplace=True)

    if year == 2007:
        df['Total'].loc[48]=df['Total'].loc[48].split()[0]
        df['Total'].loc[53]=df['Total'].loc[53].split()[1]

    elif year == 2006:

            df['Total'].loc[46]=df['Total'].loc[46].split()[0]
            df['Total'].loc[52]=df['Total'].loc[52].split()[1]
        
        
    elif year == 2002:
        
                df.loc[46 ,['Non Plan Revenue','Non Plan Capital','Non Plan Total','Total']]=[1406.44,-56.40,1350.04,2080.04]
                df.loc[47, ['Plan Capital','Plan Total','Non Plan Revenue','Total']] = [5390,5390,'...',5390]
                df.loc[48]=['Grand Total',70313.19,43186.81,113500.00,270169.60,26639.87,296809.47,410309.47]

    
    df['Year']=str(year) + '-' + str(year+1)[2:]
    
    return df

# For 2007,2006,2002 (2008 is missing)


In [ ]:
def table_to_df_6(year):
    print(year)
    file = "ministry_budget_provision/es" + str(year) + '-' + str(year+1)[2:] + ".pdf"
    tables = tabula.read_pdf(file, pages = "all")
    tabula.convert_into(file, "expenditure_summary/dummy_table.csv",pages="all")
    df=pd.read_csv("expenditure_summary/dummy_table.csv",encoding= 'ISO-8859-1',names=["Ministry", "Plan Revenue", "Plan Capital", "Plan Total","Non Plan Revenue", "Non Plan Capital", "Non Plan Total", "Total","Page No."],skiprows=3)


    req_index=list(df[df['Plan Revenue'].str.isupper() & ~df['Plan Revenue'].isnull()].index)

    for index in req_index:
        df.loc[index] =df.loc[index].shift(periods=-1)


    df.dropna(subset=['Ministry'],inplace=True)
    df.drop(df.loc[~df['Ministry'].str.isupper()].index, inplace=True)


    cols=list(df.columns.drop("Ministry"))
    NAN_list=list(df.loc[df[cols].isnull().all(axis=1)].index)

    for n_index in NAN_list:
                try:
                    df.loc[n_index+1]['Ministry']=df.loc[n_index]['Ministry'] + " " + df.loc[n_index+1]['Ministry']
                except:
                    df.loc[n_index-1]['Ministry']=df.loc[n_index-1]['Ministry'] + " " + df.loc[n_index]['Ministry']


    df.drop(NAN_list,inplace=True)
    df.reset_index(inplace=True)
    df.drop(['index'],1,inplace=True)
    
    df['PC Len']=0
    for element in df['Plan Capital']:
        df['PC Len'][df['Plan Capital']==element]=len(element.split())

    # Cleaning 1

    df['Non Plan Capital Temp']=np.where((df['PC Len']==2) & (df['Non Plan Capital'].isnull()),df['Non Plan Revenue'],df['Non Plan Capital'])
    df['Non Plan Revenue']=np.where((df['PC Len']==2) & (df['Non Plan Capital'].isnull()),df['Plan Total'],df['Non Plan Revenue'])
    df['Non Plan Capital']=df['Non Plan Capital Temp']
    df.drop('Non Plan Capital Temp',1, inplace=True)

    df['Total Temp']=np.where((df['PC Len']==2) & (df['Total'].isnull()) ,df['Non Plan Total'],df['Total'])
    df['Non Plan Total']=np.where((df['PC Len']==2) & (df['Total'].isnull()) ,df['Non Plan Capital'],df['Non Plan Total'])
    df['Non Plan Capital']=np.where((df['PC Len']==2) & (df['Total'].isnull()) ,df['Non Plan Revenue'],df['Non Plan Capital'])
    df['Non Plan Revenue']=np.where((df['PC Len']==2) & (df['Total'].isnull()) ,df['Plan Total'],df['Non Plan Revenue'])

    df['Total']=df['Total Temp']
    df.drop('Total Temp',1, inplace=True)



    df['Plan Total']=np.where(df['PC Len']==2 ,df['Plan Capital'].str.split().str[1],df['Plan Total'])
    df['Plan Capital']=np.where(df['PC Len']==2 ,df['Plan Capital'].str.split().str[0],df['Plan Capital'])

    # Cleaning 2

    df['Plan Revenue']=np.where(df['PC Len']==1,df['Plan Capital'],df['Plan Revenue'])
    df['Plan Capital']=np.where(df['PC Len']==1,df['Plan Total'].str.split().str[0],df['Plan Capital'])
    df['Plan Total']=np.where(df['PC Len']==1,df['Plan Total'].str.split().str[1],df['Plan Total'])


    df['Non Plan Total Temp']=np.where((df['PC Len']==1) & (df['Non Plan Total'].isnull()),df['Non Plan Capital'].str.split().str[1],df['Non Plan Total'])
    df['Non Plan Capital']=np.where((df['PC Len']==1) & (df['Non Plan Total'].isnull()),df['Non Plan Capital'].str.split().str[0],df['Non Plan Total'])
    df['Non Plan Total']=df['Non Plan Total Temp']
    df.drop('Non Plan Total Temp',1,inplace=True)

    if year == 2005:

        df.drop(['Page No.','PC Len'],1,inplace=True)
        df['Total'].loc[46]=df['Total'].loc[46].split()[0]
        df['Total'].loc[51]=df['Total'].loc[51].split()[1]
        df.loc[52]=['Grand Total',115981.49,27515.29,143496.78,330530.61, 40316.41, 370847.02, 514343.80]

    elif year == 2004:
        
        df.drop(['Page No.','PC Len'],1,inplace=True)
        df.loc[48]=['MINISTRY OF WATER RESOURCES',537.82, 42.18, 580.00, 231.63, 17.60, 294.23, 829.23]
        df.loc[49]=['MINISTRY OF YOUTH AFFAIRS AND SPORTS',385.79, 14.21, 400.00, 65.97, 0.03, 66.00, 466.00]
        df.loc[50]=['RAILWAYS **', '...', 6919.00, 6919.00, '...', '...', '...', 6919.00]
        df.loc[52]=['Grand Total', 91843.48, 53746.87, 145590.25, 293649.87, 38588.92, 332238.79, 477829.04]

    elif year == 2003:

        df['Non Plan Total Temp']=np.where(df['Non Plan Total'].isnull(),df['Total'],df['Non Plan Total'])
        df['Total']=np.where(df['Non Plan Total'].isnull(),df['Page No.'],df['Total'])
        df['Non Plan Total']=df['Non Plan Total Temp']

        df.drop(['Non Plan Total Temp','Page No.','PC Len'],1,inplace=True)

        df['Total'].loc[44]=df['Total'].loc[44].split()[0]
        
    df['Year']=str(year) + '-' + str(year+1)[2:]
        
    return df
    
# For 2003,2004,2005

In [ ]:

def table_to_df_7(year):
    print(year)
    file = "ministry_budget_provision/es" + str(year) + '-' + str(year+1)[2:] + ".xlsx"
    df=pd.read_excel(file ,names=["Ministry", "Plan Revenue", "Plan Capital", "Plan Total","Non Plan Revenue", "Non Plan Capital", "Non Plan Total", "Total","Page No."],skiprows=3)





    df.dropna(subset=['Ministry'],inplace=True)
    df.drop(df.loc[~df['Ministry'].str.isupper()].index, inplace=True)


    cols=list(df.columns.drop("Ministry"))
    NAN_list=list(df.loc[df[cols].isnull().all(axis=1)].index)

    for n_index in NAN_list:
            try:
                df.loc[n_index+1]['Ministry']=df.loc[n_index]['Ministry'] + " " + df.loc[n_index+1]['Ministry']
            except:
                df.loc[n_index-1]['Ministry']=df.loc[n_index-1]['Ministry'] + " " + df.loc[n_index]['Ministry']


    df.drop(NAN_list,inplace=True)
    df.reset_index(inplace=True)
    df.drop(['index','Page No.'],1,inplace=True)

    df.fillna("...",inplace=True)
      
    df['Year']=str(year) + '-' + str(year+1)[2:]
        
    return df
    
# For 2000, 2013,2015

In [ ]:
def removing_cols(df):
    df.fillna("...")
    df.replace({"...":0},inplace=True)
    df.replace({"..":0},inplace=True)
    df.replace({"…":0},inplace=True)
    df.replace({'...   ':0},inplace=True)
    df.replace({'...  ':0},inplace=True)
    df.replace({' ...   ':0},inplace=True)
   
    df['Revenue']=df['Plan Revenue'].astype('float') + df['Non Plan Revenue'].astype('float')
    df['Capital']=df['Plan Capital'].astype('float') + df['Non Plan Capital'].astype('float')
#   df.drop(['Plan Revenue','Plan Capital','Plan Total','Non Plan Revenue','Non Plan Capital','Non Plan Total'],1,inplace=True)
    
    df= df[['Ministry','Revenue','Capital','Total','Year']]
    return df


In [ ]:
df_1= table_to_df_1(2001)
df_1= removing_cols(df_1)

df_2= pd.concat([table_to_df_2(i) for i in [2017,2018,2019]])

df_3= pd.concat([table_to_df_3(i) for i in [2011,2012,2014,2016]])
df_3= removing_cols(df_3)

df_4= pd.concat([table_to_df_4(i) for i in [2009,2010]])
df_4= removing_cols(df_4)

df_5= pd.concat([table_to_df_5(i) for i in [2002,2006,2007]])
df_5= removing_cols(df_5)

df_6= pd.concat([table_to_df_6(i) for i in [2003,2004,2005]])
df_6= removing_cols(df_6)

df_7= pd.concat([table_to_df_7(i) for i in [2000,2013,2015]])
df_7= removing_cols(df_7)


final_df=pd.concat([df_1,df_2,df_3,df_4,df_5,df_6,df_7])

final_df.to_csv("ministry_budget_provision/expenditure_summary_all.csv",index=False)

##### Extra Cleaning: Manual

- RENAMED HOUSING AFFAIRS AND URBAN POVERTY ALLEIVIATION, URBAN DEVELOPMENT into MINISTRY OF HOUSING AND URBAN AFFAIRS


- Combined MINISTRY OF CULTURE, MINISTRY OF TOURISM into MINISTRY OF CULTURE AND TOURISM


- Combined MINISTRY OF COALS, MINISTRY OF MINES into MINISTRY OF COALS AND MINES


- Renamed MINISTRY OF ENIVIRONMENT AND FORESTS into ENVIRONMENT, FORESTS AND CLIMATE CHANGE


- Renamed MINISTRY OF SMALL SCALE INDUSTRIES into MINISTRY OF MICRO, SMALL AND MEDIUM ENTERPRISES


- Renamed MINISTRY OF NON-CONVENTIONAL ENERGY SOURCES into MINISTRY OF NEW AND RENEWABLE ENERGY


- Combined MINISTRY OF WATER RESOURCES into MINISTRY OF WATER RESOURCES, RIVER DEVELOPMENT AND GANGA REJUVENATION


**Resulting File:** "expenditure_summary_cleaned.csv

In [ ]:

cleaned_df=pd.read_csv("ministry_budget_provision/expenditure_summary_cleaned.csv")
cleaned_df.replace("...",0,inplace=True)

num_cols=cleaned_df.columns
num_cols=num_cols.drop(['Ministry','Year'])
for col in num_cols:
        cleaned_df[col]=pd.to_numeric(cleaned_df[col])


        
cleaned_df.reset_index(inplace=True)
cleaned_df.drop("index",1,inplace=True)        
cleaned_df.to_csv("ministry_budget_provision/ministry_expenditure_cleaned.csv",index=False)